In [ ]:
import tensorflow as tf
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import time
import sys

import numpy as np
import tensorflow as tf

#params
height = 50
width = 60
rollouts = 3000
bar = 2
bals = 1
speed = 1
verbose = True
learning_rate = 0.01
learning_rate_update = 0.999

np.random.seed(0)
DTYPE = tf.float32

current_game = tf.placeholder(dtype=DTYPE,shape=(height, width),name='game')
input_label = tf.placeholder(dtype=DTYPE, shape=(3), name='input_label')
reward_p = tf.placeholder(dtype=DTYPE, name='reward')
learning_rate_p = tf.placeholder(dtype=DTYPE, name='learning_rate')

reshaped = tf.reshape(current_game, (1,height * width))
W1 = tf.Variable(tf.random_normal([height * width, height * width], stddev=0.1), name="w1", dtype=DTYPE)
W2 = tf.Variable(tf.random_normal([height * width, 3], stddev=0.1), name="w2", dtype=DTYPE)

l1 = tf.nn.relu(tf.matmul(reshaped, W1))
l2 = tf.matmul(l1, W2)
p = tf.nn.softmax(tf.squeeze(l2))

loss = tf.nn.softmax_cross_entropy_with_logits(labels=input_label, logits=p)
grad = tf.gradients(loss, [W1, W2])

W1 += learning_rate * -1 * reward_p * grad[0]
W2 += learning_rate * -1 * reward_p * grad[1]

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

def init(bals):
    pos = []
    for j in range(bals):
        x_start = np.random.randint(round(height*0.25), round(height*0.75))
        y_start = np.random.randint(round(width*0.25), round(width*0.75))
        position = np.array([x_start, y_start])
        speed = np.array([1 if np.random.randint(2) == 1 else -1, 
                 1 if np.random.randint(2) == 1 else -1])
        pos.append([position, speed])

    player1 = np.random.randint(height)
    player2 = np.random.randint(height)
    
    return pos, player1, player2

def update_ball(position, speed, a1, a2):
    if position[0] <= 0:
        speed[0] = np.abs(speed[0])
    if position[0] >= height - 1:
        speed[0] = -np.abs(speed[0])
    if position[1] <= 0:
        if speed[0] > 0 and a1 == 2:
            speed[0] +=1
        if speed[0] < 0 and a1 == 0:
            speed[0] -=1
        if speed[0] == 0 and a1 == 2:
            speed[0] = 1
        if speed[0] == 0 and a1 == 0:
            apeed[0] = -1
            
        speed[1] = np.abs(speed[1])
    if position[1] >= width - 1:
        if speed[0] > 0 and a2 == 2:
            speed[0] +=1
        if speed[0] < 0 and a2 == 0:
            speed[0] -=1
        if speed[0] == 0 and a2 == 2:
            speed[0] = 1
        if speed[0] == 0 and a2 == 0:
            apeed[0] = -1
        
        speed[1] = -np.abs(speed[1])
    position+=speed
    position[0] = position[0].clip(0, height-1)
    position[1] = position[1].clip(0, width-1)
    return [position, speed]
    
def update_player1(pos, player1):
    return generic_player(pos, player1, 0)

def update_player2(g):
    probs = sess.run(p, feed_dict = {current_game : g})
    return np.random.choice(np.arange(3), p=probs)
    
def get_matrix(g, pos, player1, player2):
    g = g*0
    for v in pos:
        g[v[0][0], v[0][1]] = 2
    g[player1 - bar: player1 + bar, 0] = 2
    g[player2 - bar: player2 + bar, width -1] = 2
    return g
    
def generic_player(pos, player, player_pos):
    # First we check which ball we should pay attention to
    dist = np.Inf
    for v in pos:
        if np.abs(player_pos - v[0][1]) < dist:
            position = v[0]
            dist = np.abs(player_pos - v[0][1])
    if position[0] > player:
        return 2
    elif position[0] < player:
        return 0
    else:
        return 1

def update_player(player, action):
    if action == 0:
        player-=1
    elif action == 2:
        player+=1
    if player < bar:
        return bar
    if player > height - bar - 1:
        return height - bar - 1
    return player
    
def check_game(pos, speed, player1, player2):
    # Check player 1
    for v in pos:
        position = v[0]
        if position[1] <= 0:
            if position[0] > player1 + bar or position[0] < player1 - bar:
                points[1] += 1
                return 1
    
    # Check player 2:
    for v in pos:
        position = v[0]
        if position[1] >= width - 1:
            if position[0] > player2 + bar or position[0] < player2 - bar:
                points[0] += 1
                return -1

    return 0

plot = False

# Play the game
pos, player1, player2 = init(bals)
g = np.zeros((height, width))
a0_map = np.array([1,0,0])
a1_map = np.array([0,1,0])
a2_map = np.array([0,0,1])

all_points = []

if plot:
    fig,ax = plt.subplots(1,1)
points = np.zeros(2)

for rollout in range(rollouts):
    actions = []
    play_mat = []
    reward = 0
    while True:
        cg = check_game(pos, speed, player1, player2)
        if cg != 0:
            reward = cg
            pos, player1, player2 = init(bals)
            if verbose:
                sys.stdout.write("\r" + "Score: {} {}".format(int(points[0]), int(points[1])))
                sys.stdout.flush()
            all_points.append(np.copy(points))
            break

        a1 = update_player1(pos, player1)
        a2 = update_player2(g)
        for v in pos:
            v = update_ball(v[0], v[1], a1, a2)

        if a2 ==0:
            actions.append(a0_map)
        elif a2 == 1:
            actions.append(a1_map)
        elif a2 == 2:
            actions.append(a2_map)
        
        play_mat.append(g)
        player1 = update_player(player1, a1)
        player2 = update_player(player1, a2)
        g = get_matrix(g, pos, player1, player2)

        if plot:
            ax.clear()
            ax.imshow(g, cmap='Greys') 
            ax.set_title("Score: {} {}".format(int(points[0]), int(points[1])))
            fig.canvas.draw()
    
    if reward != 0:
        learning_rate = learning_rate * learning_rate_update
        if False:
            sys.stdout.write("\rUpdating the weights with reward {}".format(reward))
            sys.stdout.flush()
        # We update the weights
        for counter in range(len(actions)):
            sess.run([W1, W2], feed_dict={current_game: play_mat[counter], input_label: actions[counter], reward_p: reward,
                                         learning_rate_p:learning_rate})


Score: 129 53

KeyboardInterrupt: 

In [ ]:
from pylab import *
plot([x[0] for x in all_points])
plot([x[1] for x in all_points])
grid()
legend(['generic player', 'learner'])
title('Score')
xlabel('Iterations')